# Data Augmentation
Michael's Version

We will try a number of trivial strategies for augmenting data.

## 1. Identity Transformation
To help model the types of data in the input and output space, let's add rows for every unique source and target sentence where the change is empty, and repeat the sentence back. We can do this for the training sentences and the `Source` values from the dev sentences.

In [16]:
import pandas as pd 

langs = ['bribri', 'maya', 'guarani']

def create_identity_dataframe(lang):
    train_df = pd.read_csv(f"../data/yoyodyne/{lang}-train.tsv", delimiter="\t")
    sentences = list(set(train_df['Source'].values.tolist() + train_df['Target'].values.tolist()))

    dev_df = pd.read_csv(f"../data/yoyodyne/{lang}-dev.tsv", delimiter="\t")
    sentences += list(set(dev_df['Source'].values.tolist()))
    return pd.DataFrame({'Source': sentences, 'Target': sentences, 'Change': 'NOCHANGE'})

for lang in langs:
    identity_df = create_identity_dataframe(lang)
    identity_df.to_csv(f"../data/augmented/{lang}-identity.tsv", sep="\t", index=False)